# Experimentos

In [1]:
# imports
import numpy as np
import pandas as pd

# time
import time
from datetime import timedelta

# Metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# ETL
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Oversampling Methods
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

# Deep Learning Framework - Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv2D, AveragePooling2D, Flatten
from keras.utils import to_categorical
from keras.backend import clear_session
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.utils import plot_model

# K-fold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

models = pd.DataFrame()

Using TensorFlow backend.


# Data

In [2]:
## Data
df = pd.read_csv('data/train.csv')

## Remove colunas constantes
colsToRemove = []
for col in df.columns:
    if df[col].std() == 0:
        colsToRemove.append(col)
df.drop(colsToRemove, axis=1, inplace=True)

## Remove colunas multiplas
colsToRemove = []
columns = df.columns
for i in range(len(columns)-1):
    print(i, end="\r")
    v = df[columns[i]]
    for j in range(i+1,len(columns)):
        m = df.loc[(df[columns[j]]*v) != 0, [columns[j], columns[i]]]
        if m.shape[0] > 0:
            c = m.iloc[0,0]/m.iloc[0,1]
            if np.array_equal(c*v.values,df[columns[j]].values):
                colsToRemove.append(columns[j])
df.drop(colsToRemove, axis=1, inplace=True)

## Remove duplicate rows
df.drop_duplicates(inplace=True)

## Split (X,Y) 
x_data = df.drop(['ID','TARGET'], axis=1)
y_data = df['TARGET'].copy()

x_data.shape

(76020, 287)

# Oversampling: Random Oversampling

## 2 CNN 1 AVG RO

In [3]:
# info
model_name = '2 CNN 1 AVG RO Clean'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = RandomOverSampler(random_state=42)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 17, 17, 1)
    x_test = x_test.reshape(x_test.shape[0], 17, 17, 1)
    x_val = x_val.reshape(x_val.shape[0], 17, 17, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(17, 17, 1), activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(243, activation='relu'))
    model.add(Dense(243, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97598 samples, validate on 13684 samples
Epoch 1/10000
97598/97598 [==============================] - 27s - loss: 0.5869 - acc: 0.6970 - val_loss: 0.5572 - val_acc: 0.7304
Epoch 2/10000
97598/97598 [==============================] - 14s - loss: 0.5247 - acc: 0.7418 - val_loss: 0.4695 - val_acc: 0.7852
Epoch 3/10000
97598/97598 [==============================] - 14s - loss: 0.5048 - acc: 0.7522 - val_loss: 0.4880 - val_acc: 0.7722
Epoch 4/10000
97598/97598 [==============================] - 14s - loss: 0.4907 - acc: 0.7606 - val_loss: 0.4972 - val_acc: 0.7409
Epoch 5/10000
97598/97598 [==============================] - 14s - loss: 0.4763 - acc: 0.7678 - val_loss: 0.4496 - val_acc: 0.7892
Epoch 6/10000
97598/97598 [==============================] - 15s - loss: 0.4644 - acc: 0.7771 - val_loss: 0.5176 - val_acc: 0.7463
Epoch 7/10000
97598/97598 [==============================] - 15s - loss: 0.4526 - acc: 0.7833 - val_loss: 0.5219 - val_acc: 0.7378
Epoch 8/10000
97598/97598 [=

97468/97468 [==============================] - 15s - loss: 0.3261 - acc: 0.8507 - val_loss: 0.4623 - val_acc: 0.8272
Epoch 32/10000
97468/97468 [==============================] - 15s - loss: 0.3247 - acc: 0.8505 - val_loss: 0.4987 - val_acc: 0.7948
Epoch 33/10000
97468/97468 [==============================] - 15s - loss: 0.3232 - acc: 0.8532 - val_loss: 0.4499 - val_acc: 0.8473
Epoch 34/10000
97468/97468 [==============================] - 15s - loss: 0.3191 - acc: 0.8541 - val_loss: 0.5096 - val_acc: 0.7955
Epoch 35/10000
97468/97468 [==============================] - 14s - loss: 0.3163 - acc: 0.8554 - val_loss: 0.4800 - val_acc: 0.8068
Epoch 36/10000
97152/97468 [============================>.] - ETA: 0sKF_5
Train on 97566 samples, validate on 13684 samples
Epoch 1/10000
97566/97566 [==============================] - 15s - loss: 0.5892 - acc: 0.6967 - val_loss: 0.4951 - val_acc: 0.7757
Epoch 2/10000
97566/97566 [==============================] - 15s - loss: 0.5227 - acc: 0.7435 - val_

97616/97616 [==============================] - 14s - loss: 0.4222 - acc: 0.8010 - val_loss: 0.4464 - val_acc: 0.7917
Epoch 10/10000
97616/97616 [==============================] - 14s - loss: 0.4122 - acc: 0.8064 - val_loss: 0.4750 - val_acc: 0.7746
Epoch 11/10000
97616/97616 [==============================] - 14s - loss: 0.4025 - acc: 0.8116 - val_loss: 0.4728 - val_acc: 0.7868
Epoch 12/10000
97616/97616 [==============================] - 15s - loss: 0.3963 - acc: 0.8153 - val_loss: 0.5207 - val_acc: 0.7550
Epoch 13/10000
97616/97616 [==============================] - 14s - loss: 0.3985 - acc: 0.8152 - val_loss: 0.4438 - val_acc: 0.7966
Epoch 14/10000
97616/97616 [==============================] - 15s - loss: 0.3821 - acc: 0.8219 - val_loss: 0.4886 - val_acc: 0.7858
Epoch 15/10000
97616/97616 [==============================] - 14s - loss: 0.3750 - acc: 0.8257 - val_loss: 0.4859 - val_acc: 0.7990
Epoch 16/10000
97616/97616 [==============================] - 15s - loss: 0.3703 - acc: 0.8

In [4]:
print(model_name)

metrics_kf

2 CNN 1 AVG RO Clean


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.932015,0.743109,0.743109,0.544625,0.697849,0.533271,0.792845,00:06:50.968997
KF_2,0.931874,0.765632,0.768954,0.553249,0.740148,0.544941,0.794555,00:07:50.548620
KF_3,0.937517,0.732480,0.732480,0.546507,0.693312,0.540896,0.808603,00:08:35.310424
KF_4,0.940776,0.743066,0.743066,0.556769,0.709560,0.562956,0.836753,00:09:17.370350
KF_5,0.938268,0.726095,0.726095,0.547093,0.687094,0.544602,0.818074,00:08:18.818434
KF_6,0.949175,0.753072,0.753072,0.547611,0.713419,0.536886,0.792160,00:12:38.427585
KF_7,0.919487,0.789906,0.789906,0.550983,0.729825,0.541529,0.793081,00:05:55.239006
KF_8,0.942613,0.714096,0.714096,0.549448,0.696016,0.548289,0.819916,00:10:04.094521
KF_9,0.936557,0.746035,0.746035,0.550795,0.705309,0.549344,0.817656,00:08:03.017807
KF_10,0.942350,0.720530,0.723863,0.552168,0.708482,0.552094,0.820681,00:07:00.381075


## 2 CNN 2 AVG RO

In [5]:
# info
model_name = '2 CNN 2 AVG RO Clean'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = RandomOverSampler(random_state=42)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 17, 17, 1)
    x_test = x_test.reshape(x_test.shape[0], 17, 17, 1)
    x_val = x_val.reshape(x_val.shape[0], 17, 17, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(17, 17, 1), activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(94, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97554 samples, validate on 13684 samples
Epoch 1/10000
97554/97554 [==============================] - 15s - loss: 0.6770 - acc: 0.5862 - val_loss: 0.6448 - val_acc: 0.6907
Epoch 2/10000
97554/97554 [==============================] - 14s - loss: 0.6245 - acc: 0.6770 - val_loss: 0.6126 - val_acc: 0.6801
Epoch 3/10000
97554/97554 [==============================] - 14s - loss: 0.6101 - acc: 0.6872 - val_loss: 0.5954 - val_acc: 0.6764
Epoch 4/10000
97554/97554 [==============================] - 15s - loss: 0.6059 - acc: 0.6892 - val_loss: 0.6696 - val_acc: 0.6132
Epoch 5/10000
97554/97554 [==============================] - 14s - loss: 0.6030 - acc: 0.6924 - val_loss: 0.5887 - val_acc: 0.6752
Epoch 6/10000
97554/97554 [==============================] - 15s - loss: 0.6018 - acc: 0.6927 - val_loss: 0.6040 - val_acc: 0.6645
Epoch 7/10000
97554/97554 [==============================] - 14s - loss: 0.5990 - acc: 0.6933 - val_loss: 0.6296 - val_acc: 0.6548
Epoch 8/10000
97554/97554 [=

97462/97462 [==============================] - 14s - loss: 0.5032 - acc: 0.7504 - val_loss: 0.5376 - val_acc: 0.7021
Epoch 23/10000
97462/97462 [==============================] - 14s - loss: 0.5024 - acc: 0.7503 - val_loss: 0.4874 - val_acc: 0.7442
Epoch 24/10000
97462/97462 [==============================] - 15s - loss: 0.5017 - acc: 0.7506 - val_loss: 0.5352 - val_acc: 0.7291
Epoch 25/10000
97462/97462 [==============================] - 15s - loss: 0.4989 - acc: 0.7524 - val_loss: 0.5217 - val_acc: 0.7055
Epoch 26/10000
97462/97462 [==============================] - 14s - loss: 0.4979 - acc: 0.7522 - val_loss: 0.5426 - val_acc: 0.7105
Epoch 27/10000
97216/97462 [============================>.] - ETA: 0sKF_4
Train on 97506 samples, validate on 13684 samples
Epoch 1/10000
97506/97506 [==============================] - 15s - loss: 0.6499 - acc: 0.6514 - val_loss: 0.6258 - val_acc: 0.7142
Epoch 2/10000
97506/97506 [==============================] - 14s - loss: 0.6252 - acc: 0.6791 - val_

97536/97536 [==============================] - 15s - loss: 0.4968 - acc: 0.7548 - val_loss: 0.5306 - val_acc: 0.7089
Epoch 22/10000
97536/97536 [==============================] - 14s - loss: 0.4955 - acc: 0.7564 - val_loss: 0.5031 - val_acc: 0.7439
Epoch 23/10000
97536/97536 [==============================] - 14s - loss: 0.4944 - acc: 0.7557 - val_loss: 0.5417 - val_acc: 0.6888
Epoch 24/10000
97536/97536 [==============================] - 14s - loss: 0.4925 - acc: 0.7572 - val_loss: 0.4847 - val_acc: 0.7463
Epoch 25/10000
97536/97536 [==============================] - 14s - loss: 0.4926 - acc: 0.7565 - val_loss: 0.5478 - val_acc: 0.7349
Epoch 26/10000
97536/97536 [==============================] - 14s - loss: 0.4925 - acc: 0.7568 - val_loss: 0.4945 - val_acc: 0.7256
Epoch 27/10000
97536/97536 [==============================] - 14s - loss: 0.4901 - acc: 0.7581 - val_loss: 0.5708 - val_acc: 0.7099
Epoch 28/10000
97536/97536 [==============================] - 14s - loss: 0.4876 - acc: 0.7

97542/97542 [==============================] - 14s - loss: 0.5005 - acc: 0.7533 - val_loss: 0.4479 - val_acc: 0.8023
Epoch 15/10000
97542/97542 [==============================] - 15s - loss: 0.4982 - acc: 0.7557 - val_loss: 0.4575 - val_acc: 0.7736
Epoch 16/10000
97542/97542 [==============================] - 14s - loss: 0.4968 - acc: 0.7550 - val_loss: 0.5253 - val_acc: 0.7482
Epoch 17/10000
97542/97542 [==============================] - 15s - loss: 0.4951 - acc: 0.7582 - val_loss: 0.4716 - val_acc: 0.7868
Epoch 18/10000
97542/97542 [==============================] - 14s - loss: 0.4940 - acc: 0.7580 - val_loss: 0.5159 - val_acc: 0.7578
Epoch 19/10000
97542/97542 [==============================] - 14s - loss: 0.4917 - acc: 0.7599 - val_loss: 0.4823 - val_acc: 0.7554
Epoch 20/10000
97542/97542 [==============================] - 14s - loss: 0.4892 - acc: 0.7597 - val_loss: 0.5844 - val_acc: 0.7227
Epoch 21/10000
97542/97542 [==============================] - 14s - loss: 0.4879 - acc: 0.7

97498/97498 [==============================] - 14s - loss: 0.4832 - acc: 0.7637 - val_loss: 0.4660 - val_acc: 0.7528
Epoch 56/10000
97498/97498 [==============================] - 14s - loss: 0.4829 - acc: 0.7641 - val_loss: 0.4964 - val_acc: 0.7409
Epoch 57/10000
97498/97498 [==============================] - 14s - loss: 0.4819 - acc: 0.7639 - val_loss: 0.5045 - val_acc: 0.7371
Epoch 58/10000
97498/97498 [==============================] - 14s - loss: 0.4823 - acc: 0.7639 - val_loss: 0.4934 - val_acc: 0.7369
Epoch 59/10000
97498/97498 [==============================] - 15s - loss: 0.4813 - acc: 0.7656 - val_loss: 0.5063 - val_acc: 0.7363
Epoch 60/10000
97498/97498 [==============================] - 14s - loss: 0.4819 - acc: 0.7652 - val_loss: 0.5269 - val_acc: 0.7353
Epoch 61/10000
97498/97498 [==============================] - 15s - loss: 0.4798 - acc: 0.7651 - val_loss: 0.4970 - val_acc: 0.7477
Epoch 62/10000
97498/97498 [==============================] - 14s - loss: 0.4785 - acc: 0.7

In [6]:
print(model_name)

metrics_kf

2 CNN 2 AVG RO Clean


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.854939,0.801783,0.801783,0.542243,0.738641,0.498428,0.709062,00:16:56.977970
KF_2,0.830432,0.818187,0.821509,0.545877,0.745058,0.513917,0.736683,00:08:36.934877
KF_3,0.839952,0.813425,0.813425,0.549034,0.746492,0.527217,0.760852,00:06:51.614001
KF_4,0.851605,0.815342,0.815342,0.547977,0.749542,0.521061,0.748356,00:11:08.734708
KF_5,0.849342,0.795547,0.795547,0.541554,0.718923,0.509529,0.738490,00:12:52.797813
KF_6,0.863832,0.788612,0.791935,0.538422,0.720967,0.488938,0.696527,00:17:20.575127
KF_7,0.865330,0.803409,0.803409,0.543332,0.724488,0.514625,0.746119,00:13:09.021140
KF_8,0.843965,0.791082,0.791082,0.540533,0.728655,0.496165,0.708235,00:07:22.230283
KF_9,0.860947,0.791884,0.795217,0.544170,0.740521,0.508639,0.728720,00:24:34.564344
KF_10,0.861141,0.785989,0.785989,0.542163,0.729836,0.505457,0.726615,00:19:04.735515


# Oversampling: SMOTE

## 2 CNN 1 AVG SM

In [7]:
# info
model_name = '2 CNN 1 AVG SM Clean'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 17, 17, 1)
    x_test = x_test.reshape(x_test.shape[0], 17, 17, 1)
    x_val = x_val.reshape(x_val.shape[0], 17, 17, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(17, 17, 1), activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(243, activation='relu'))
    model.add(Dense(243, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97520 samples, validate on 13684 samples
Epoch 1/10000
97520/97520 [==============================] - 14s - loss: 0.5000 - acc: 0.7740 - val_loss: 0.5714 - val_acc: 0.6982
Epoch 2/10000
97520/97520 [==============================] - 14s - loss: 0.3653 - acc: 0.8490 - val_loss: 0.4383 - val_acc: 0.8376
Epoch 3/10000
97520/97520 [==============================] - 14s - loss: 0.3313 - acc: 0.8645 - val_loss: 0.2340 - val_acc: 0.8913
Epoch 4/10000
97520/97520 [==============================] - 14s - loss: 0.3129 - acc: 0.8733 - val_loss: 0.3875 - val_acc: 0.8309
Epoch 5/10000
97520/97520 [==============================] - 14s - loss: 0.3019 - acc: 0.8784 - val_loss: 0.3795 - val_acc: 0.8309
Epoch 6/10000
97520/97520 [==============================] - 14s - loss: 0.2937 - acc: 0.8817 - val_loss: 0.4128 - val_acc: 0.8253
Epoch 7/10000
97520/97520 [==============================] - 14s - loss: 0.2861 - acc: 0.8854 - val_loss: 0.3031 - val_acc: 0.8621
Epoch 8/10000
97520/97520 [=

97562/97562 [==============================] - 14s - loss: 0.2472 - acc: 0.9029 - val_loss: 0.3937 - val_acc: 0.8281
Epoch 18/10000
97562/97562 [==============================] - 14s - loss: 0.2457 - acc: 0.9034 - val_loss: 0.3282 - val_acc: 0.8521
Epoch 19/10000
97562/97562 [==============================] - 14s - loss: 0.2432 - acc: 0.9053 - val_loss: 0.2637 - val_acc: 0.8774
Epoch 20/10000
97562/97562 [==============================] - 14s - loss: 0.2409 - acc: 0.9059 - val_loss: 0.2157 - val_acc: 0.9008
Epoch 21/10000
97562/97562 [==============================] - 14s - loss: 0.2394 - acc: 0.9065 - val_loss: 0.3114 - val_acc: 0.8627
Epoch 22/10000
97562/97562 [==============================] - 14s - loss: 0.2385 - acc: 0.9072 - val_loss: 0.3675 - val_acc: 0.8529
Epoch 23/10000
97562/97562 [==============================] - 14s - loss: 0.2343 - acc: 0.9078 - val_loss: 0.3121 - val_acc: 0.8664
Epoch 24/10000
97562/97562 [==============================] - 14s - loss: 0.2368 - acc: 0.9

97520/97520 [==============================] - 14s - loss: 0.3504 - acc: 0.8575 - val_loss: 0.3845 - val_acc: 0.8377
Epoch 3/10000
97520/97520 [==============================] - 14s - loss: 0.3215 - acc: 0.8692 - val_loss: 0.3234 - val_acc: 0.8688
Epoch 4/10000
97520/97520 [==============================] - 14s - loss: 0.3062 - acc: 0.8770 - val_loss: 0.4266 - val_acc: 0.7995
Epoch 5/10000
97520/97520 [==============================] - 14s - loss: 0.2956 - acc: 0.8819 - val_loss: 0.5029 - val_acc: 0.7835
Epoch 6/10000
97520/97520 [==============================] - 14s - loss: 0.2842 - acc: 0.8860 - val_loss: 0.3369 - val_acc: 0.8522
Epoch 7/10000
97520/97520 [==============================] - 14s - loss: 0.2801 - acc: 0.8874 - val_loss: 0.3456 - val_acc: 0.8488
Epoch 8/10000
97520/97520 [==============================] - 14s - loss: 0.2749 - acc: 0.8902 - val_loss: 0.2713 - val_acc: 0.8749
Epoch 9/10000
97520/97520 [==============================] - 14s - loss: 0.2712 - acc: 0.8921 - v

In [8]:
print(model_name)

metrics_kf

2 CNN 1 AVG SM Clean


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.966701,0.752187,0.752187,0.556725,0.688914,0.566740,0.852164,00:05:47.182295
KF_2,0.961022,0.758665,0.761987,0.561483,0.717549,0.571771,0.845982,00:06:51.560074
KF_3,0.968851,0.761041,0.761041,0.567338,0.701610,0.584864,0.870429,00:06:31.339220
KF_4,0.966044,0.787312,0.787312,0.559221,0.705620,0.569076,0.847540,00:06:06.343293
KF_5,0.970844,0.749758,0.749758,0.564996,0.689759,0.581846,0.872139,00:09:54.951289
KF_6,0.968501,0.769226,0.769226,0.576158,0.717415,0.598293,0.879374,00:11:53.722201
KF_7,0.970546,0.777936,0.777936,0.562542,0.711613,0.574889,0.852933,00:11:11.823218
KF_8,0.968057,0.754527,0.754527,0.558952,0.678303,0.572432,0.865430,00:08:00.218848
KF_9,0.966987,0.774189,0.774189,0.569926,0.698249,0.589417,0.877779,00:07:47.287148
KF_10,0.958692,0.720110,0.720110,0.536645,0.690680,0.504316,0.740429,00:06:16.249074


## 2 CNN 2 AVG SM

In [9]:
# info
model_name = '2 CNN 2 AVG SM Clean'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 17, 17, 1)
    x_test = x_test.reshape(x_test.shape[0], 17, 17, 1)
    x_val = x_val.reshape(x_val.shape[0], 17, 17, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(17, 17, 1), activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(94, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97502 samples, validate on 13684 samples
Epoch 1/10000
97502/97502 [==============================] - 15s - loss: 0.5868 - acc: 0.7291 - val_loss: 0.5886 - val_acc: 0.7049
Epoch 2/10000
97502/97502 [==============================] - 14s - loss: 0.5234 - acc: 0.7727 - val_loss: 0.5369 - val_acc: 0.6980
Epoch 3/10000
97502/97502 [==============================] - 14s - loss: 0.4407 - acc: 0.8063 - val_loss: 0.3906 - val_acc: 0.8204
Epoch 4/10000
97502/97502 [==============================] - 14s - loss: 0.3884 - acc: 0.8320 - val_loss: 0.4891 - val_acc: 0.7950
Epoch 5/10000
97502/97502 [==============================] - 15s - loss: 0.3721 - acc: 0.8404 - val_loss: 0.3923 - val_acc: 0.8300
Epoch 6/10000
97502/97502 [==============================] - 14s - loss: 0.3640 - acc: 0.8453 - val_loss: 0.3473 - val_acc: 0.8518
Epoch 7/10000
97502/97502 [==============================] - 14s - loss: 0.3560 - acc: 0.8500 - val_loss: 0.3883 - val_acc: 0.8286
Epoch 8/10000
97502/97502 [=

97382/97382 [==============================] - 15s - loss: 0.3581 - acc: 0.8461 - val_loss: 0.3808 - val_acc: 0.8492
Epoch 15/10000
97382/97382 [==============================] - 14s - loss: 0.3476 - acc: 0.8546 - val_loss: 0.3486 - val_acc: 0.8491
Epoch 16/10000
97382/97382 [==============================] - 14s - loss: 0.3435 - acc: 0.8556 - val_loss: 0.3073 - val_acc: 0.8728
Epoch 17/10000
97382/97382 [==============================] - 14s - loss: 0.3395 - acc: 0.8582 - val_loss: 0.2847 - val_acc: 0.8737
Epoch 18/10000
97382/97382 [==============================] - 14s - loss: 0.3382 - acc: 0.8583 - val_loss: 0.3619 - val_acc: 0.8558
Epoch 19/10000
97382/97382 [==============================] - 15s - loss: 0.3355 - acc: 0.8589 - val_loss: 0.2961 - val_acc: 0.8735
Epoch 20/10000
97382/97382 [==============================] - 14s - loss: 0.3348 - acc: 0.8597 - val_loss: 0.2540 - val_acc: 0.8882
Epoch 21/10000
97382/97382 [==============================] - 14s - loss: 0.3332 - acc: 0.8

97589/97589 [==============================] - 14s - loss: 0.3621 - acc: 0.8475 - val_loss: 0.4740 - val_acc: 0.8046
Epoch 8/10000
97589/97589 [==============================] - 14s - loss: 0.3577 - acc: 0.8485 - val_loss: 0.4038 - val_acc: 0.8337
Epoch 9/10000
97589/97589 [==============================] - 15s - loss: 0.3499 - acc: 0.8523 - val_loss: 0.3220 - val_acc: 0.8541
Epoch 10/10000
97589/97589 [==============================] - 14s - loss: 0.3448 - acc: 0.8556 - val_loss: 0.3833 - val_acc: 0.8337
Epoch 11/10000
97589/97589 [==============================] - 14s - loss: 0.3423 - acc: 0.8566 - val_loss: 0.3152 - val_acc: 0.8595
Epoch 12/10000
97589/97589 [==============================] - 14s - loss: 0.3389 - acc: 0.8582 - val_loss: 0.4543 - val_acc: 0.8064
Epoch 13/10000
97589/97589 [==============================] - 14s - loss: 0.3368 - acc: 0.8591 - val_loss: 0.3326 - val_acc: 0.8558
Epoch 14/10000
97589/97589 [==============================] - 14s - loss: 0.3330 - acc: 0.860

97459/97459 [==============================] - 14s - loss: 0.3058 - acc: 0.8759 - val_loss: 0.4078 - val_acc: 0.8367
Epoch 42/10000
97459/97459 [==============================] - 14s - loss: 0.3045 - acc: 0.8770 - val_loss: 0.2524 - val_acc: 0.8842
Epoch 43/10000
97459/97459 [==============================] - 14s - loss: 0.3049 - acc: 0.8765 - val_loss: 0.3432 - val_acc: 0.8533
Epoch 44/10000
97459/97459 [==============================] - 14s - loss: 0.3041 - acc: 0.8776 - val_loss: 0.3803 - val_acc: 0.8215
Epoch 45/10000
97459/97459 [==============================] - 15s - loss: 0.3031 - acc: 0.8781 - val_loss: 0.4135 - val_acc: 0.8285
Epoch 46/10000
97459/97459 [==============================] - 14s - loss: 0.3031 - acc: 0.8773 - val_loss: 0.2922 - val_acc: 0.8707
Epoch 47/10000
97459/97459 [==============================] - 14s - loss: 0.3021 - acc: 0.8779 - val_loss: 0.3206 - val_acc: 0.8613
Epoch 48/10000
97459/97459 [==============================] - 15s - loss: 0.3015 - acc: 0.8

In [10]:
print(model_name)

metrics_kf

2 CNN 2 AVG SM Clean


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.944088,0.783302,0.786624,0.556739,0.692391,0.566189,0.849665,00:09:49.641799
KF_2,0.942728,0.803478,0.806800,0.563042,0.733443,0.572254,0.839800,00:09:09.442240
KF_3,0.947303,0.793021,0.793021,0.559961,0.722712,0.567478,0.837543,00:08:09.882062
KF_4,0.943069,0.799553,0.802875,0.567436,0.712793,0.583995,0.864378,00:10:20.654107
KF_5,0.945054,0.792545,0.792545,0.562827,0.703547,0.576561,0.858853,00:07:03.258030
KF_6,0.948148,0.802411,0.802411,0.550485,0.720714,0.543286,0.800053,00:15:14.904128
KF_7,0.946988,0.805678,0.805678,0.565499,0.720672,0.579420,0.855038,00:12:46.157997
KF_8,0.947326,0.782505,0.782505,0.556894,0.691406,0.566671,0.850829,00:09:44.047610
KF_9,0.946999,0.790644,0.790644,0.569867,0.722040,0.587504,0.865149,00:16:00.162210
KF_10,0.945990,0.797626,0.800960,0.560595,0.724394,0.568696,0.838969,00:07:09.157154


# Metrics

In [11]:
models

,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
2 CNN 1 AVG RO Clean,0.937063,0.743402,0.744068,0.549925,0.708102,0.545481,0.809432,00:08:27.417681
2 CNN 2 AVG RO Clean,0.852149,0.800526,0.801524,0.543530,0.734312,0.508397,0.729966,00:13:47.818577
2 CNN 1 AVG SM Clean,0.966625,0.760495,0.760827,0.561399,0.699971,0.571364,0.850420,00:08:02.067666
2 CNN 2 AVG SM Clean,0.945769,0.795076,0.796406,0.561335,0.714411,0.571205,0.846028,00:10:32.730733
